# Using FAIRshake API

This notebook walks through using FAIRshake API with coreapi's python implementation, it works just as easily in javascript or any language with a coreapi implementation. Given that swagger is also exposed at <https://fairshake.cloud/api/?format=openapi>, a swagger-based client can *also* be used.

For more information, refer to the documentation at https://fairshake.cloud/api/coreapi/

Dependencies:
`pip install coreapi-cli`

In [1]:
import coreapi

client = coreapi.Client()

url = 'http://localhost:8000'

schema = client.get(url + '/v2/coreapi/')

## Step 1. Authentication

If you don't already have a user registered with FAIRshake, you need to create one. This can be done manually at <https://fairshake.cloud/accounts/signup/> or, through the API itself, this returns the API key for your account.

**Note**: Authentication is only required if you plan on *creating*/*changing* things. Read-access is available without authentication.

In [2]:
# Set these to create/login to your own account
your_username = 'test'
your_password = 'my_test_pass'
# Set to True if you've already registered
registered = True

In [3]:
if not registered:
    results = client.action(
        schema,
        ['auth', 'registration', 'create'],
        dict(
            username=your_username,
            password1=your_password,
            password2=your_password,
        ),
    )
    registered = True

If you already have an account, you can login through the API as well to obtain your API key.

In [4]:
results = client.action(
    schema,
    ['auth', 'login', 'create'],
    dict(
        username=your_username,
        password=your_password,
    )
)
api_key = results['key']
results

OrderedDict([('key', 'fbc6f7f16f2adbb18181ac8852b9674450d519da')])

Now that we have our API key, we can reinstantiate our Client with an authenticated transport layer.

In [5]:
client = coreapi.Client(auth=coreapi.auth.TokenAuthentication(token=api_key, scheme='token'))
schema = client.get(url + '/v2/coreapi/')

We can test that it worked by reading information about the logged in user.

In [6]:
client.action(
    schema,
    ['auth', 'user', 'read'],
)

OrderedDict([('pk', 2),
             ('username', 'test'),
             ('email', ''),
             ('first_name', ''),
             ('last_name', '')])

## Step 2. Project, Digital Object, Rubric, Metric Management

All elements expose themselves in the same way with a common set of attributes for search and identification with a few extra attributes distinguishing each element.

Here, quite simply, is the gist of these data models:

```python
class Identifiable:
  id: int
  url: str
  title: str
  description: str
  image: str
  tags: str
  type: ['', 'any', 'data', 'repo', 'test', 'tool']
  authors: Author[]

class Project(Identifiable):
  digital_objects: DigitalObject[]

class DigitalObject(Identifiable):
  rubrics: Rubric[]

class Rubric(Identifiable):
  license: str
  metrics: Metric[]

class Metric(Identifiable):
  license: str
  rationale: str
  principle: str
  fairmetrics: str
  fairsharing: str
```

Queries can be made by providing any of the parameters and we'll return the subset of the database which satisfies those parameter constraints. Though you use `title=something` we do a fuzzy search if it makes sense to do so. More find-tuned queries are actually supported by the API but not yet documented which would allow for django-style filters e.g. `title__contains=something`.

**Note**: Results are paginated; use `params={'page': n}` to go through pages

In [7]:
# List all projects
client.action(schema, ['project', 'list'])

OrderedDict([('count', 9),
             ('next', None),
             ('previous', None),
             ('results',
              [OrderedDict([('id', 1),
                            ('uri', None),
                            ('minid', 'http://minid/000000'),
                            ('url', 'http://my-objects.com'),
                            ('title', 'My Project'),
                            ('description', 'Project is great'),
                            ('image', None),
                            ('tags', 'my project test'),
                            ('authors', [2]),
                            ('digital_objects', [1])]),
               OrderedDict([('id', 2),
                            ('uri', 'a'),
                            ('minid', 'a'),
                            ('url', 'http://maayanlab.com'),
                            ('title', 'a'),
                            ('description', ''),
                            ('image', ''),
                            ('tags',

In [8]:
# List all Digital objects of type Tool
client.action(schema, ['digital_object', 'list'], params=dict(type='tool'))

OrderedDict([('count', 0),
             ('next', None),
             ('previous', None),
             ('results', [])])

In [9]:
# Create a digital object
obj = client.action(schema, ['digital_object', 'create'], params=dict(
  minid='http://minid/00001',
  url='http://my-objects.com/00001',
  title='My Object',
  description='Object is great',
  tags='my object test',
  type='tool',
))
obj_id = obj['id']
obj

OrderedDict([('id', 9),
             ('uri', None),
             ('minid', 'http://minid/00001'),
             ('url', 'http://my-objects.com/00001'),
             ('title', 'My Object'),
             ('description', 'Object is great'),
             ('image', None),
             ('tags', 'my object test'),
             ('type', 'tool'),
             ('authors', [2]),
             ('rubrics', [])])

In [10]:
# Create a project
proj = client.action(schema, ['project', 'create'], params=dict(
  minid='http://minid/000000',
  url='http://my-objects.com',
  title='My Project',
  description='Project is great',
  tags='my project test',
  digital_objects=[obj_id],
))
proj_id = proj['id']
proj

OrderedDict([('id', 10),
             ('uri', None),
             ('minid', 'http://minid/000000'),
             ('url', 'http://my-objects.com'),
             ('title', 'My Project'),
             ('description', 'Project is great'),
             ('image', None),
             ('tags', 'my project test'),
             ('authors', [2]),
             ('digital_objects', [9])])

In [11]:
# Create a metric
metric = client.action(schema, ['metric', 'create'], params=dict(
  title='My Metric',
  description='It has a url',
  type='url',
  tags='my project test',
  license='MIT',
  rationale='https://fairrationals.com/test',
  principle='https://fairprincipals.com/test',
))
metric_id = metric['id']
metric

OrderedDict([('id', 9),
             ('uri', None),
             ('minid', None),
             ('url', None),
             ('title', 'My Metric'),
             ('description', 'It has a url'),
             ('image', None),
             ('tags', 'my project test'),
             ('type', 'url'),
             ('license', 'MIT'),
             ('rationale', 'https://fairrationals.com/test'),
             ('principle', 'https://fairprincipals.com/test'),
             ('fairmetrics', None),
             ('fairsharing', None),
             ('authors', [2])])

In [12]:
# Create a rubric
rubric = client.action(schema, ['rubric', 'create'], params=dict(
  title='My Rubric',
  description='Rubric is great',
  tags='my project test',
  type='tool',
  license='MIT',
  metrics=[
      metric_id,
  ],
))
rubric_id = rubric['id']
rubric

OrderedDict([('id', 9),
             ('uri', None),
             ('minid', None),
             ('url', None),
             ('title', 'My Rubric'),
             ('description', 'Rubric is great'),
             ('image', None),
             ('tags', 'my project test'),
             ('type', 'tool'),
             ('license', 'MIT'),
             ('authors', [2]),
             ('metrics', [9])])

In [13]:
# Associate digital object with a rubric (Future)
# client.action(schema, ['digital_object', 'partial_update'], params=dict(
#   id=obj_id,
#   rubrics=[rubric_id],
# ))

# Performing Assessments

```python
class Assessment:
  project: Project
  target: DigitalObject
  rubric: Rubric
  methodology: ['self', 'user', 'auto', 'test']
  answers: Answer[]

class Answer:
  metric: Metric
  answer: str
  comment: str
  url_comment: str
```

In [14]:
# Create an assessment
client.action(schema, ['assessment', 'create'], params=dict(
  project=proj_id,
  target=obj_id,
  rubric=rubric_id,
  methodology='test',
  answers=[
      {
          'metric': metric_id,
          'answer': 'http://my_url.com',
      },
  ],
))

OrderedDict([('id', 11),
             ('project', 10),
             ('target', 9),
             ('rubric', 9),
             ('answers',
              [OrderedDict([('id', 10),
                            ('answer', 'http://my_url.com'),
                            ('comment', None),
                            ('url_comment', None),
                            ('assessment', 11),
                            ('metric', 9)])]),
             ('methodology', 'test'),
             ('requestor', None),
             ('assessor', 'test'),
             ('timestamp', '2018-07-27T17:36:14.946277Z')])

## Obtaining the score of a Digital Object

In [15]:
score = client.action(schema, ['score', 'list'], params=dict(target=obj_id))
score

OrderedDict([('9', OrderedDict([('9', 1.0)]))])

## Displaying FAIR insignia

The insignia client library exposes a function, `build_svg` which accepts a container and the score dict.

**Note**: tooltip information is written into the svg and can be instantiated with your favorite tooltip library (e.g. https://getbootstrap.com/docs/4.1/components/tooltips/)

In [60]:
import json
from IPython.display import HTML
HTML('''
<input
    id="url-var"
    type="text"
    style="display: none;"
    value="{url}">
<textarea
    id="score-var"
    type="text"
    style="display: none;">
    {score}</textarea>
<div
    id="insignia-container-1"
    data-target="{obj_id}"
    style="width: 40px; height: 40px; border: 0px solid black" />
'''.format(url=url, obj_id=obj_id, score=json.dumps(score)))

In [61]:
%%javascript
require.config({
  paths: {
    jquery: "https://code.jquery.com/jquery-3.3.1.min",
  }
})
require(['jquery'], function($) {
    $(function() {
        var url = $('#url-var').val()
        var score = JSON.parse($('#score-var').val())
        require([
            url + '/v2/static/scripts/insignia.js',
        ], function(insignia) {
            insignia.build_svg('#insignia-container-1', score)
        })
    })
})

<IPython.core.display.Javascript object>